# Busca por gênero de filmes no IMDB

O IMDB é uma plataforma dedicada a informações sobre filmes e séries de tv, e disponibiliza alguns [datasets](https://developer.imdb.com/non-commercial-datasets/) para uso não comercial em seu site. 

## Preparando os arquivos

In [1]:
import pandas as pd
import requests
import os
import gzip
import shutil

In [2]:
# URL do arquivo .gz
url = 'https://datasets.imdbws.com/title.basics.tsv.gz'

# Caminho onde os arquivos deverão ser salvos
pasta = '/Users/bentofh/Documents/GitHub/ancine/IMDB'

In [3]:
# Definir as variáveis necessárias
caminho_arquivo_gz = os.path.join(pasta, 'title.basics.tsv.gz')
caminho_arquivo_tsv = os.path.join(pasta, 'title.basics.tsv')
caminho_arquivo_parquet = os.path.join(pasta, 'imdb.parquet')

# Baixar o arquivo .gz
response = requests.get(url, stream=True)
with open(caminho_arquivo_gz, 'wb') as f:
    f.write(response.content)

# Extrair o arquivo .tsv
with gzip.open(caminho_arquivo_gz, 'rb') as f_in:
    with open(caminho_arquivo_tsv, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

# Remover o arquivo .gz
os.remove(caminho_arquivo_gz)

# Ler o arquivo .tsv em um DataFrame
df_imdb = pd.read_csv(caminho_arquivo_tsv, delimiter='\t', low_memory=False)

# Filtrar os filmes lançados a partir de 2010
# Certifique-se de que 'startYear' é uma coluna numérica ou converta-a
df_imdb['startYear'] = pd.to_numeric(df_imdb['startYear'], errors='coerce')
df_imdb = df_imdb[df_imdb['startYear'] >= 2013]

# Manter apenas as colunas originalTitle e genres
df_imdb = df_imdb[['originalTitle', 'genres']]

# Manter apenas o primeiro gênero na coluna genres
df_imdb['genres'] = df_imdb['genres'].apply(lambda x: x.split(',')[0] if pd.notnull(x) and ',' in x else x)

# Salvar o DataFrame como um arquivo Parquet
df_imdb.to_parquet(caminho_arquivo_parquet)

# Remover o arquivo .tsv se não for mais necessário
os.remove(caminho_arquivo_tsv)

# Limpar o df utilizado para o trabalho
del df_imdb